# Install YOLOv5 Dependencies

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5 

In [1]:
#Run this cell for initial set up on YOLOv5 PyTorch

# !git clone https://github.com/ultralytics/yolov5
%cd yolov5
# !pip install -r requirements.txt 

#make sure you ar on /yolov5 directories
%pwd

C:\Users\user\Documents\SKRIPSI\Coba\yolov5


'C:\\Users\\user\\Documents\\SKRIPSI\\Coba\\yolov5'

In [ ]:
!pip install --upgrade pip

In [11]:
# install dependencies as necessary
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import time

# to display images
from IPython.display import Image, clear_output

In [ ]:
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## Dataset preparation

Make sure that the directory is outside the current directory of yolov5

In [ ]:
# Run this cell for initial setup on Roboflow
# pip install roboflow --user

from roboflow import Roboflow

In [ ]:
# import Dataset from roboflow

rf = Roboflow(api_key="WYy7UTxLiklRVM4VChB0")
project = rf.workspace("wound").project("wound-detection")
dataset = project.version(4).download("yolov5")

In [ ]:
# pip list
os.listdir()

In [ ]:
# Try on pre trained model Run this cell if you want to know

# Download pre trained model from torch hub and ultralytics yolov5

# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Test to create prediction from pretrained model on one data

# img = 'https://img.jakpost.net/c/2020/10/31/2020_10_31_106644_1604140506._large.jpg'
# results = model(img)
# results.print()
# results.save()

### REAL TIME DETECTION

A real time detection using openCV VideoCapture frame as prediction data

In [ ]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# TRAIN ON CUSTOM MODEL

## Data and configuration preparation

In [7]:
# define number of classes based on data.yaml there is one class on this dataset which is wound
import yaml

with open("../Wound-detection-4/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
num_classes

In [6]:
# customize iPython writefile
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [11]:
# Create custom configuration of related models. The models configuration pytorch provides store in /yolov5/models copy
# the file and set your own configuration for hyper parameter tunning

# YOLOv5s model configuration

%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: 1  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

SyntaxError: invalid syntax (<ipython-input-11-841f66f1d33f>, line 14)

## Training

#### Parameters definitions:

- img: refers to the input images size.
- batch: refers to the batch size (number of training examples utilized in one iteration).
- epochs: refers to the number of training epochs. An epoch corresponds to one cycle through the full training dataset.
- data: refers to the path to the yaml file.
- cfg: define the model configuration.

In [12]:
os.listdir()

['.dockerignore',
 '.git',
 '.gitattributes',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 'classify',
 'CONTRIBUTING.md',
 'data',
 'detect.py',
 'export.py',
 'hubconf.py',
 'LICENSE',
 'models',
 'README.md',
 'requirements.txt',
 'setup.cfg',
 'train.py',
 'tutorial.ipynb',
 'utils',
 'val.py']

In [8]:
%time
!python train.py --img 416 --batch 16 --epochs 3 --data ../Wound-detection-4/data.yaml --cfg models/yolov5s.yaml --weights ../yolov5s.pt --cache

Wall time: 0 ns


### Evaluate the training model

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
# Image(filename='runs/train/exp/results.png', width=1000)


# from object_detection.utils.plots import plot_results
plot_results('runs/train/exp/results.csv')